# Baseline CNN Model - Dogs vs Cats

## Experiment 001: Transfer Learning with ResNet

This notebook implements a baseline CNN model using transfer learning for the Dogs vs Cats classification problem.

### Approach:
- Use pretrained ResNet50 model
- 5-fold stratified cross-validation
- Binary classification (dog vs cat)
- Evaluation metric: Log Loss
- Generate predictions for test set submission

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU available, using CPU")

PyTorch version: 2.2.0+cu118
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB
GPU Memory: 85.1 GB


In [2]:
# Data paths
DATA_DIR = '/home/data'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

# Verify data exists
print(f"Train directory exists: {os.path.exists(TRAIN_DIR)}")
print(f"Test directory exists: {os.path.exists(TEST_DIR)}")

# List some training images
train_files = os.listdir(TRAIN_DIR)[:5]
print(f"Sample training files: {train_files}")

Train directory exists: True
Test directory exists: True
Sample training files: ['dog.5.jpg', 'cat.8112.jpg', 'cat.1197.jpg', 'dog.8491.jpg', 'dog.9129.jpg']


In [3]:
# Create dataset class
class DogsCatsDataset(Dataset):
    def __init__(self, file_paths, labels=None, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        return image

# Data transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = val_transform

In [4]:
# Prepare training data
print("Loading training data...")
train_files = [os.path.join(TRAIN_DIR, f) for f in os.listdir(TRAIN_DIR) if f.endswith('.jpg')]
train_labels = [1 if 'dog' in os.path.basename(f) else 0 for f in train_files]

print(f"Total training images: {len(train_files)}")
print(f"Dog images: {sum(train_labels)}")
print(f"Cat images: {len(train_labels) - sum(train_labels)}")

# Convert to numpy arrays
train_files = np.array(train_files)
train_labels = np.array(train_labels)

Loading training data...
Total training images: 22500
Dog images: 11258
Cat images: 11242


In [5]:
# Create model function
def create_model():
    # Load pretrained ResNet50
    model = models.resnet50(pretrained=True)
    
    # Freeze early layers
    for param in model.parameters():
        param.requires_grad = False
    
    # Replace final layer for binary classification
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, 1),
        nn.Sigmoid()
    )
    
    return model

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=5):
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device).float()
            
            optimizer.zero_grad()
            output = model(data).squeeze()
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if batch_idx % 100 == 0:
                print(f'Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}')
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device).float()
                output = model(data).squeeze()
                loss = criterion(output, target)
                val_loss += loss.item()
                
                val_preds.extend(output.cpu().numpy())
                val_targets.extend(target.cpu().numpy())
        
        # Calculate metrics
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        val_log_loss = log_loss(val_targets, val_preds)
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'  Train Loss: {train_loss:.4f}')
        print(f'  Val Loss: {val_loss:.4f}')
        print(f'  Val Log Loss: {val_log_loss:.4f}')
        
        scheduler.step(val_loss)
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            
    return model, val_log_loss

In [6]:
# Cross-validation setup
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Store results
cv_scores = []
oof_predictions = np.zeros(len(train_files))

print(f"\nStarting {n_splits}-fold cross-validation...\n")

Using device: cuda

Starting 5-fold cross-validation...



In [7]:
# Run cross-validation
fold = 1
for train_idx, val_idx in skf.split(train_files, train_labels):
    print(f"\n{'='*50}")
    print(f"FOLD {fold}/{n_splits}")
    print(f"{'='*50}")
    
    # Split data
    X_train, X_val = train_files[train_idx], train_files[val_idx]
    y_train, y_val = train_labels[train_idx], train_labels[val_idx]
    
    print(f"Training samples: {len(X_train)}")
    print(f"Validation samples: {len(X_val)}")
    
    # Create datasets
    train_dataset = DogsCatsDataset(X_train, y_train, transform=train_transform)
    val_dataset = DogsCatsDataset(X_val, y_val, transform=val_transform)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
    
    # Create model
    model = create_model()
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)
    
    # Train model
    model, val_score = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=3)
    
    # Store predictions
    model.eval()
    val_preds = []
    with torch.no_grad():
        for data, _ in val_loader:
            data = data.to(device)
            output = model(data).squeeze()
            val_preds.extend(output.cpu().numpy())
    
    oof_predictions[val_idx] = val_preds
    cv_scores.append(val_score)
    
    print(f"Fold {fold} Log Loss: {val_score:.4f}")
    
    fold += 1

# Overall CV score
print(f"\n{'='*50}")
print(f"CROSS-VALIDATION RESULTS")
print(f"{'='*50}")
print(f"Mean Log Loss: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")
print(f"Individual folds: {cv_scores}")


FOLD 1/5
Training samples: 18000
Validation samples: 4500


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/nonroot/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

  0%|          | 416k/97.8M [00:00<00:24, 4.22MB/s]

  5%|▍         | 4.66M/97.8M [00:00<00:03, 27.8MB/s]

...

Epoch 3, Batch 300, Loss: 0.0903


Epoch 3, Batch 400, Loss: 0.0387


Epoch 3, Batch 500, Loss: 0.3858


Epoch 3/3:
  Train Loss: 0.0949
  Val Loss: 0.0467
  Val Log Loss: 0.0467


Fold 5 Log Loss: 0.0467

CROSS-VALIDATION RESULTS
Mean Log Loss: 0.0479 ± 0.0043
Individual folds: [0.044094325701767995, 0.05371179078450418, 0.05216084367308546, 0.04278605528187914, 0.04670264739165255]


In [8]:
# Generate predictions on test set
print("\nGenerating predictions on test set...")

# Load test files
test_files = [os.path.join(TEST_DIR, f) for f in sorted(os.listdir(TEST_DIR)) if f.endswith('.jpg')]
test_ids = [int(os.path.splitext(os.path.basename(f))[0]) for f in test_files]

print(f"Total test images: {len(test_files)}")

# Create test dataset and loader
test_dataset = DogsCatsDataset(test_files, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Average predictions from all folds
test_predictions = np.zeros(len(test_files))

fold = 1
for train_idx, val_idx in skf.split(train_files, train_labels):
    print(f"Generating predictions from fold {fold}...")
    
    # Recreate and load model for this fold
    model = create_model()
    model = model.to(device)
    
    # Create temporary training split to train model
    X_train, X_val = train_files[train_idx], train_files[val_idx]
    y_train, y_val = train_labels[train_idx], train_labels[val_idx]
    
    train_dataset = DogsCatsDataset(X_train, y_train, transform=train_transform)
    val_dataset = DogsCatsDataset(X_val, y_val, transform=val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)
    
    # Train model
    model, _ = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=3)
    
    # Generate predictions
    model.eval()
    fold_preds = []
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            output = model(data).squeeze()
            fold_preds.extend(output.cpu().numpy())
    
    test_predictions += np.array(fold_preds)
    fold += 1

# Average predictions across folds
test_predictions /= n_splits

print(f"Test predictions shape: {test_predictions.shape}")
print(f"Test predictions range: [{test_predictions.min():.4f}, {test_predictions.max():.4f}]")


Generating predictions on test set...
Total test images: 2500
Generating predictions from fold 1...


Epoch 1, Batch 0, Loss: 0.7391


Epoch 1, Batch 100, Loss: 0.1232


Epoch 1, Batch 200, Loss: 0.0847


Epoch 1, Batch 300, Loss: 0.1244


...

Epoch 3, Batch 300, Loss: 0.3463


Epoch 3, Batch 400, Loss: 0.0802


Epoch 3, Batch 500, Loss: 0.1305


Epoch 3/3:
  Train Loss: 0.0957
  Val Loss: 0.0456
  Val Log Loss: 0.0456


Test predictions shape: (2500,)
Test predictions range: [0.0000, 1.0000]


In [9]:
# Create submission file
print("\nCreating submission file...")

submission = pd.DataFrame({
    'id': test_ids,
    'label': test_predictions
})

# Sort by id
submission = submission.sort_values('id').reset_index(drop=True)

print(f"Submission shape: {submission.shape}")
print(f"Sample predictions:")
print(submission.head(10))

# Save submission
SUBMISSION_DIR = '/home/submission'
os.makedirs(SUBMISSION_DIR, exist_ok=True)
submission_path = os.path.join(SUBMISSION_DIR, 'submission.csv')
submission.to_csv(submission_path, index=False)

print(f"Submission saved to: {submission_path}")

# Verify submission format
sample_sub = pd.read_csv('/home/data/sample_submission.csv')
print(f"\nSubmission format matches sample: {list(submission.columns) == list(sample_sub.columns)}")
print(f"ID ranges match: {submission['id'].min() == sample_sub['id'].min() and submission['id'].max() == sample_sub['id'].max()}")


Creating submission file...
Submission shape: (2500, 2)
Sample predictions:
   id     label
0   1  0.999861
1   2  0.000488
2   3  0.856156
3   4  0.579628
4   5  0.000111
5   6  0.999985
6   7  0.996927
7   8  0.001029
8   9  0.000088
9  10  0.975234
Submission saved to: /home/submission/submission.csv

Submission format matches sample: True
ID ranges match: True
